##pyspark 실습
### - csv 파일 불러오기
-  pandas
-  sc.textFile() 및 전처리
-  sqlContext()

  

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 198 kB 53.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=4dd9f5eaf75be199aa070325d9107d951e416dfc93993d089dadec91fdcdc653
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


[Transformation 명령어 더 알아보기](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations)

In [6]:
!java -version

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


요런 csv 파일을 확인했다고 생각을 해봅시다!

In [15]:
import pandas as pd
csv_path = '/content/sample_data/train.csv'
df= pd.read_csv(csv_path)
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Spark로 (빅)데이터를 불러와서 가공하여 봅시다.

In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [10]:
# csv_path = '/content/sample_data/train.csv'
csv_data_0 = sc.textFile(csv_path)
csv_data_0.take(5)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']

> list 형태로 반환이 되는군요! 이것을 ,에 따라 `split`해볼께요.



In [12]:
# 비어있는 라인은 제외하고, delimeter인 ,로 line을 분리해 줍니다. 
csv_data_1 = csv_data_0.filter(lambda line: len(line)>1).map(lambda line: line.split(","))   
csv_data_1.take(5)

[['PassengerId',
  'Survived',
  'Pclass',
  'Name',
  'Sex',
  'Age',
  'SibSp',
  'Parch',
  'Ticket',
  'Fare',
  'Cabin',
  'Embarked'],
 ['1',
  '0',
  '3',
  '"Braund',
  ' Mr. Owen Harris"',
  'male',
  '22',
  '1',
  '0',
  'A/5 21171',
  '7.25',
  '',
  'S'],
 ['2',
  '1',
  '1',
  '"Cumings',
  ' Mrs. John Bradley (Florence Briggs Thayer)"',
  'female',
  '38',
  '1',
  '0',
  'PC 17599',
  '71.2833',
  'C85',
  'C'],
 ['3',
  '1',
  '3',
  '"Heikkinen',
  ' Miss. Laina"',
  'female',
  '26',
  '0',
  '0',
  'STON/O2. 3101282',
  '7.925',
  '',
  'S'],
 ['4',
  '1',
  '1',
  '"Futrelle',
  ' Mrs. Jacques Heath (Lily May Peel)"',
  'female',
  '35',
  '1',
  '0',
  '113803',
  '53.1',
  'C123',
  'S']]

> Column만 따로 분리해 볼까요?

In [13]:
col = csv_data_1.take(1)
col

[['PassengerId',
  'Survived',
  'Pclass',
  'Name',
  'Sex',
  'Age',
  'SibSp',
  'Parch',
  'Ticket',
  'Fare',
  'Cabin',
  'Embarked']]

> 칼럼 이외의 데이터도 분리해 보겠습니다. 첫 원소가 숫자형태인 것을 이용해 보겠습니다!!

In [17]:
csv_data_2 = csv_data_1.filter(lambda line: line[0].isdecimal())
csv_data_2.take(5)

[['1',
  '0',
  '3',
  '"Braund',
  ' Mr. Owen Harris"',
  'male',
  '22',
  '1',
  '0',
  'A/5 21171',
  '7.25',
  '',
  'S'],
 ['2',
  '1',
  '1',
  '"Cumings',
  ' Mrs. John Bradley (Florence Briggs Thayer)"',
  'female',
  '38',
  '1',
  '0',
  'PC 17599',
  '71.2833',
  'C85',
  'C'],
 ['3',
  '1',
  '3',
  '"Heikkinen',
  ' Miss. Laina"',
  'female',
  '26',
  '0',
  '0',
  'STON/O2. 3101282',
  '7.925',
  '',
  'S'],
 ['4',
  '1',
  '1',
  '"Futrelle',
  ' Mrs. Jacques Heath (Lily May Peel)"',
  'female',
  '35',
  '1',
  '0',
  '113803',
  '53.1',
  'C123',
  'S'],
 ['5',
  '0',
  '3',
  '"Allen',
  ' Mr. William Henry"',
  'male',
  '35',
  '0',
  '0',
  '373450',
  '8.05',
  '',
  'S']]

In [29]:
len(csv_data_2.take(2)[1]), len(col[0])

(13, 12)

> 확인하니 Name에 두가지 요소가 split으로 나뉘어져 있습니다.칼럼 개수를 맞춰줍시다.

In [32]:
# Name에서 나뉘어진 두개의 element를 위해 insert
col[0].insert(4,'FullName')
col

[['PassengerId',
  'Survived',
  'Pclass',
  'Name',
  'FullName',
  'Sex',
  'Age',
  'SibSp',
  'Parch',
  'Ticket',
  'Fare',
  'Cabin',
  'Embarked']]

In [33]:
csv_data_3 = csv_data_2.map(lambda line: [(col[0][i], linedata) for i, linedata in enumerate(line)])
csv_data_3.take(5)

[[('PassengerId', '1'),
  ('Survived', '0'),
  ('Pclass', '3'),
  ('Name', '"Braund'),
  ('FullName', ' Mr. Owen Harris"'),
  ('Sex', 'male'),
  ('Age', '22'),
  ('SibSp', '1'),
  ('Parch', '0'),
  ('Ticket', 'A/5 21171'),
  ('Fare', '7.25'),
  ('Cabin', ''),
  ('Embarked', 'S')],
 [('PassengerId', '2'),
  ('Survived', '1'),
  ('Pclass', '1'),
  ('Name', '"Cumings'),
  ('FullName', ' Mrs. John Bradley (Florence Briggs Thayer)"'),
  ('Sex', 'female'),
  ('Age', '38'),
  ('SibSp', '1'),
  ('Parch', '0'),
  ('Ticket', 'PC 17599'),
  ('Fare', '71.2833'),
  ('Cabin', 'C85'),
  ('Embarked', 'C')],
 [('PassengerId', '3'),
  ('Survived', '1'),
  ('Pclass', '3'),
  ('Name', '"Heikkinen'),
  ('FullName', ' Miss. Laina"'),
  ('Sex', 'female'),
  ('Age', '26'),
  ('SibSp', '0'),
  ('Parch', '0'),
  ('Ticket', 'STON/O2. 3101282'),
  ('Fare', '7.925'),
  ('Cabin', ''),
  ('Embarked', 'S')],
 [('PassengerId', '4'),
  ('Survived', '1'),
  ('Pclass', '1'),
  ('Name', '"Futrelle'),
  ('FullName', ' Mrs.

> 위와 같이 tuple 형태로 가공까지 성공했습니다!! 

### SQLContext 모듈을 import해서 불러오는 방법도 알아보겠습니다!!

1. `SparkContext()` 객체에 `addFile`
2. 1의 결과물을 `SQLContext`로 감싸서 객체 생성
3. 객체에서  `.read.csv`

In [34]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark import SparkFiles

url = 'https://storage.googleapis.com/tf-datasets/titanic/train.csv'
sc.addFile(url) # 웹에서 파일 불러오기
sqlContext = SQLContext(sc)

df = sqlContext.read.csv(SparkFiles.get("train.csv"), header=True, inferSchema= True)
df.show(5, truncate = False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------+------+----+------------------+-----+-------+-----+-------+-----------+-----+
|survived|sex   |age |n_siblings_spouses|parch|fare   |class|deck   |embark_town|alone|
+--------+------+----+------------------+-----+-------+-----+-------+-----------+-----+
|0       |male  |22.0|1                 |0    |7.25   |Third|unknown|Southampton|n    |
|1       |female|38.0|1                 |0    |71.2833|First|C      |Cherbourg  |n    |
|1       |female|26.0|0                 |0    |7.925  |Third|unknown|Southampton|y    |
|1       |female|35.0|1                 |0    |53.1   |First|C      |Southampton|n    |
|0       |male  |28.0|0                 |0    |8.4583 |Third|unknown|Queenstown |y    |
+--------+------+----+------------------+-----+-------+-----+-------+-----------+-----+
only showing top 5 rows



> pandas dataframe과는 다른 느낌으로 데이터를 확인할 수 있군요!
하지만 조작법은 비슷합니다.

나이가 40세 이상인 분을 필터링해볼까요?

In [35]:
df2 = df[df['age']>40]
df2.show(5,truncate = False)

+--------+------+----+------------------+-----+-------+------+-------+-----------+-----+
|survived|sex   |age |n_siblings_spouses|parch|fare   |class |deck   |embark_town|alone|
+--------+------+----+------------------+-----+-------+------+-------+-----------+-----+
|0       |male  |66.0|0                 |0    |10.5   |Second|unknown|Southampton|y    |
|0       |male  |42.0|1                 |0    |52.0   |First |unknown|Southampton|n    |
|1       |female|49.0|1                 |0    |76.7292|First |D      |Cherbourg  |n    |
|0       |male  |65.0|0                 |1    |61.9792|First |B      |Cherbourg  |n    |
|0       |male  |45.0|1                 |0    |83.475 |First |C      |Southampton|n    |
+--------+------+----+------------------+-----+-------+------+-------+-----------+-----+
only showing top 5 rows

